In [1]:
import torch
import clip
import glob
import os
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm
import torchmetrics
import torchvision

/home/tingyang/miniconda3/envs/DGS_my/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
def read_images(path):
    image_files = glob.glob(os.path.join(path, '*.png'))  # Change the file extension if needed
    image_files.sort()  # Sort the image files alphabetically
    images = []
    for file in image_files:
        image = Image.open(file)  # Read the image using OpenCV or any other library
        images.append(image)
        # print(file)
    return images


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [3]:
from torchmetrics.image import KernelInceptionDistance
KID = KernelInceptionDistance(subsets=100,subset_size=20).cuda()

/home/tingyang/miniconda3/envs/DGS_my/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `Kernel Inception Distance` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [ ]:
logdir = 'logdir'
folders = image_files = glob.glob(os.path.join(logdir, 'eval-*1'))
folders.sort()
# seqname = 'bear'
# path = f'logdir/eval-{seqname}-1/eval/'
# Load the CLIP model
cross_total = 0.
compare_total = 0.
mean_total = 0.
std_total = 0.
folder_num = 0
for folder in folders:
    path = os.path.join(folder, 'rendered')
    print(path)
    seqname = folder.split('-')[1]
    gt_path = f'datasource/{seqname}/imgs/'
    mask_path = f'datasource/{seqname}/masks/'
    gts = read_images(gt_path)
    masks = read_images(mask_path)
    imgs = read_images(path)
    # print(len(imgs))
    sim_total = 0.
    sim_num = 0
    gt_tensor = []
    render_tensor = []
    for i in tqdm(range(len(imgs))):
        
        gts[i] = np.array(gts[i])
        masks[i] = np.array(masks[i])
        gts[i][masks[i][:,:,2]<128] = 255
        gts[i] = np.concatenate([np.ones([420,1920,3],dtype=gts[i].dtype)*255,gts[i],np.ones([420,1920,3],dtype=gts[i].dtype)*255],axis=0)
        gt_tensor.append(torch.from_numpy(np.array(torchvision.transforms.Resize([512,512])(Image.fromarray(gts[i])))).cuda())
        render_tensor.append(torch.from_numpy(np.array(imgs[i])).cuda())
        gts[i] = Image.fromarray(gts[i])
        with torch.no_grad():
            gt_feature = model.encode_image(preprocess(gts[i]).unsqueeze(0).to(device))
            render_feature = model.encode_image(preprocess(imgs[i]).unsqueeze(0).to(device))
            sim = torch.nn.functional.cosine_similarity(gt_feature, render_feature)
        sim_total += sim
        sim_num += 1
    print('over')
    try:
        sim_avg = sim_total / sim_num
        print(path,':',sim_avg)
        compare_total += sim_avg
        folder_num += 1
    except:
        print('Error Calculating CLIP:',path)
    try:
        gt_tensor = torch.stack(gt_tensor,dim=0).permute(0,3,1,2).cuda()
        render_tensor = torch.stack(render_tensor,dim=0).permute(0,3,1,2).cuda()
        KID.update(gt_tensor, real=True)
        KID.update(render_tensor, real=False)
        mean, std = KID.compute()
        KID.reset()
        mean_total += mean
        std_total += std
        print(path,':',mean,' ',std)
    except:
        print('Error Calculating KID:',path)
    

clip_avg_cross = compare_total / folder_num
mean_avg = mean_total / folder_num
std_avg = std_total / folder_num
# Print the cosine distance
print(clip_avg_cross, mean_avg, std_avg)

In [32]:
logdir = 'logdir'
folders = image_files = glob.glob(os.path.join(logdir, 'eval-*1'))
folders.sort()
# seqname = 'bear'
# path = f'logdir/eval-{seqname}-1/eval/'
# Load the CLIP model
cross_total = 0.
compare_total = 0.
mean_total = 0.
std_total = 0.
folder_num = 0
for folder in folders:
    path = os.path.join(folder, 'eval')
    print(path)
    gts = read_images(path)
    # print(len(imgs))
    sim_total = 0.
    sim_num = 0
    features = []
    for i in tqdm(range(len(gts))):
        with torch.no_grad():
            gt_feature = model.encode_image(preprocess(gts[i]).unsqueeze(0).to(device))
            for f in features:
                sim = torch.nn.functional.cosine_similarity(gt_feature, f)
                sim_total += sim
                # print(sim)
                sim_num += 1
            features.append(gt_feature)
    print('over')
    try:
        sim_avg = sim_total / sim_num
        if sim_avg<0.9:
            continue
        cross_total +=sim_avg
        print(path,':',sim_avg)
        folder_num += 1
    except:
        print('Error Calculating CLIP:',path)
    
    

clip_avg_cross = cross_total / folder_num
# Print the cosine distance
print(clip_avg_cross, mean_avg, std_avg)

logdir/eval-bailang-1/eval


100%|██████████| 22/22 [00:00<00:00, 57.43it/s]


over
logdir/eval-bailang-1/eval : tensor([0.9155], device='cuda:0', dtype=torch.float16)
logdir/eval-baiyou-1/eval


100%|██████████| 8/8 [00:00<00:00, 59.00it/s]


over
logdir/eval-baiyou-1/eval : tensor([0.9478], device='cuda:0', dtype=torch.float16)
logdir/eval-bear-1/eval


...

over
logdir/eval-turtle-1/eval : tensor([0.9175], device='cuda:0', dtype=torch.float16)
logdir/eval-tusun-1/eval


100%|██████████| 9/9 [00:00<00:00, 61.76it/s]


over
logdir/eval-tusun-1/eval : tensor([0.9243], device='cuda:0', dtype=torch.float16)
logdir/eval-zongxiong-1/eval


100%|██████████| 33/33 [00:00<00:00, 53.31it/s]

over
tensor([0.9292], device='cuda:0', dtype=torch.float16) tensor(0.1189, device='cuda:0') tensor(0.0109, device='cuda:0')


In [19]:
from torch.nn.functional import interpolate
KID = KernelInceptionDistance(subset_size=20,normalize=True).cuda()
path = 'logdir/eval-panda-1/rendered/'
print(path)
gt_path = f'datasource/panda/imgs/'
mask_path = f'datasource/panda/masks/'
gts = read_images(gt_path)
masks = read_images(mask_path)
imgs = read_images(path)
# print(len(imgs))
sim_total = 0.
sim_num = 0
gt_tensor = []
render_tensor = []
for i in tqdm(range(len(imgs))):
    
    gts[i] = np.array(gts[i],dtype=np.float32)/255.
    masks[i] = np.array(masks[i],dtype=np.float32)/255.
    gts[i] = gts[i]*masks[i][:,:,2:]+(1-masks[i][:,:,2:])
    gts[i] = np.concatenate([np.ones([420,1920,3],dtype=np.float32),gts[i],np.ones([420,1920,3],dtype=np.float32)],axis=0)
    # print(gts[i].shape)
    gt_tensor.append(interpolate(torch.from_numpy(gts[i]).cuda().moveaxis(-1,0)[None,...],size=(512,512),mode='bilinear')[0].moveaxis(0,-1))
    render_tensor.append(torch.from_numpy(np.array(imgs[i],dtype=np.float32)/255.).cuda())
    
try:
    gt_tensor = torch.stack(gt_tensor,dim=0).permute(0,3,1,2).cuda()
    render_tensor = torch.stack(render_tensor,dim=0).permute(0,3,1,2).cuda()
    KID.update(gt_tensor, real=True)
    KID.update(render_tensor, real=False)
    # KID.update(gt_tensor[1::2], real=False)
    mean, std = KID.compute()
    KID.reset()
    print(path,':',mean,' ',std)
except:
    print('Error Calculating KID:',path)

logdir/eval-panda-1/rendered/


100%|██████████| 66/66 [00:10<00:00,  6.16it/s]


logdir/eval-panda-1/rendered/ : tensor(0.0725, device='cuda:0')   tensor(0.0085, device='cuda:0')


In [12]:
KID.reset()
KID.update(gt_tensor, real=True)
KID.update(render_tensor, real=False)

In [13]:
KID.compute()

(tensor(0.7259, device='cuda:0'), tensor(0.0206, device='cuda:0'))

In [ ]:
cv2.imwrite('test.png',gt_tensor[0].moveaxis(0,-1)[:,:,[2,1,0]].cpu().numpy()*255.)
cv2.imwrite('test1.png',render_tensor[0].moveaxis(0,-1)[:,:,[2,1,0]].cpu().numpy()*255.)

In [19]:
render_tensor.shape

torch.Size([66, 3, 512, 512])

In [ ]:
0.9575,0.0789,0.0083
0.9023,0.2157,0.0083